In [1]:
# import all packages
import numpy as np
import gempy as gp
import matplotlib.pyplot as plt
import pandas as pd
import pyvista as pv
from scipy.interpolate import griddata
from scipy.interpolate import Rbf
from matplotlib import cm
import pykrige
from pykrige.ok3d import OrdinaryKriging3D
from pykrige.uk3d import UniversalKriging3D
from scipy.interpolate import RBFInterpolator
from geomdl import BSpline
from geomdl import utilities as utils
import plotly.graph_objects as go
import ipywidgets
import warnings
warnings.filterwarnings("ignore")

d:\anaconda\envs\gempy\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Not subsurface compatibility available


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


#### NURBS 

In [2]:
def NURBS(left_x,right_x,mid_left_x,mid_right_x,top_left_x,top_right_x,top_x,
          left_y,right_y,mid_left_y,mid_right_y,top_left_y,top_right_y,top_y,
          nr_points=20,fig=True):
    
    # Create a 3-dimensional B-spline Curve
    curve = BSpline.Curve()

    # Set degree
    curve.degree = 3
    # Set control points
    ctrlpts = [
        [left_x, left_y], [mid_left_x, mid_left_y], [top_left_x, top_left_y],
        [top_x, top_y], [top_right_x, top_right_y], [mid_right_x, mid_right_y],
        [right_x, right_y]
    ]
    curve.ctrlpts = ctrlpts

    # Set knot vector
    curve.knotvector = [0, 0,0, *np.arange(0,1+1/(np.array(curve.ctrlpts).shape[0]-3)-0.0001,1/(np.array(curve.ctrlpts).shape[0]-3)), 1, 1, 1]
    curve.knotvector = utils.generate_knot_vector(curve.degree , len(curve.ctrlpts))
    # print(curve.knotvector)
    # Set evaluation delta (controls the number of curve points)
    curve.delta = 1/nr_points

    # Get curve points (the curve will be automatically evaluated)
    curve_points = curve.evalpts

    v=np.array([])
    for i in np.arange(0,1,curve.delta):
        v = np.append([curve.derivatives(u=i,order=1)[1][0],curve.derivatives(u=i,order=1)[1][1]],v)
    v = v.reshape(nr_points,2)
    inv = np.arctan2(v[:, 1], v[:, 0])
    degree = np.mod(np.degrees(inv), 360)
    
    if fig:
      fig,ax=plt.subplots(figsize=(7,7))
      ax.plot(np.array(curve_points)[:,0],np.array(curve_points)[:,1],"-")
      ax.plot(np.array(curve.ctrlpts)[:,0],np.array(curve.ctrlpts)[:,1],"o")
      ax.set_xlim(0,20)
      ax.set_ylim(0,20)

    return curve_points,degree

#### Create model

In [3]:

class clay:

    def __init__(self,slice_nr = 2,
                 slice_1_parameter = None,
                 slice_2_parameter = None,
                 slice_1_pos = 10,
                 slice_2_pos = 10,
                 formation = None):
        

        self.slice_nr = slice_nr

        self.slider_1, self.degree_1 = slice_1_parameter
        self.slider_2, self.degree_2= slice_2_parameter

        self.slice_pos_1 = slice_1_pos
        self.slice_pos_2 = slice_2_pos
        self.formation = formation

    def get_input_points(self):
        x_slice_points = self.slider_1
        y_slice_points = self.slider_2

        return x_slice_points,y_slice_points

    def set_input_data(self):

        x_slice_points = self.slider_1
        y_slice_points = self.slider_2
 
        # nr_1 = np.array(x_slice_points).shape[0]
        # nr_2 = np.array(y_slice_points).shape[0]

        x_slice = [np.array(x_slice_points)[:,0],
                   np.ones(len(x_slice_points))*self.slice_pos_1,
                   np.array(x_slice_points)[:,1]]
        x_slice = pd.DataFrame((np.array(x_slice).T),columns=('x','y','z'))

        y_slice = [np.ones(len(y_slice_points))*self.slice_pos_2,
                   np.array(y_slice_points)[:,0],
                   np.array(y_slice_points)[:,1]]
        y_slice = pd.DataFrame((np.array(y_slice).T),columns=('x','y','z'))

        formation = pd.DataFrame(data={'formation':[self.formation]*(len(x_slice) + len(y_slice))})

        surface = pd.concat([pd.concat([x_slice,y_slice],ignore_index=True),formation],axis=1)
        orientation_1 = pd.DataFrame(data={"dip":self.degree_1-180,"azimuth":['90']*len(x_slice),"polarity":['1']*len(x_slice)})
        orientation_2 = pd.DataFrame(data={"dip":self.degree_2-180,"azimuth":['0']*len(y_slice),"polarity":['1']*len(y_slice)})

        # orientation_1 = pd.DataFrame(data={"x":[10],"y":[10],"z":[0],"formation":['clay'],"dip":['0'],"azimuth":['0'],"polarity":['1']})

        # orientation_2 = pd.DataFrame(data={"x":[np.array(x_slice_points)[:,0][np.argmax(np.array(x_slice_points)[:,1])]],"y":[self.slice_pos_1],"z":[np.array(x_slice_points)[:,1].max()],
        #                                    "formation":['clay'],"dip":['180'],"azimuth":['0'],"polarity":['1']})
        # orientation_3 = pd.DataFrame(data={"x":[self.slice_pos_2],"y":[np.array(y_slice_points)[:,0][np.argmax(np.array(y_slice_points)[:,1])]],"z":[np.array(y_slice_points)[:,1].max()],
        #                                    "formation":['clay'],"dip":['180'],"azimuth":['0'],"polarity":['1']})
        
        orientation = pd.concat([surface,pd.concat([orientation_1,orientation_2],ignore_index=True)],axis=1)
        return orientation,surface

In [4]:
# set default value for flat clay

def choose_standard_model(model):

    if  model == 'clay_slice_1':
        left_x_1 = ipywidgets.FloatSlider(value=0,min=0.0, max=20.0,description='left x')
        right_x_1 = ipywidgets.FloatSlider(value=20,min=0.0, max=20.0,description='right x')
        mid_left_x_1 = ipywidgets.FloatSlider(value=3,min=0.0, max=20.0,description='mid left x')
        mid_right_x_1 = ipywidgets.FloatSlider(value=17,min=0.0, max=20.0,description='mid right x')
        top_left_x_1 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0,description='top left x')
        top_right_x_1 = ipywidgets.FloatSlider(value=13,min=0.0, max=20.0,description='top right x')
        top_x_1 = ipywidgets.FloatSlider(value=10,min=0.0, max=20.0,description='top x')

        left_y_1 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='left y')
        right_y_1 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='right y')
        mid_left_y_1 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='mid left y')
        mid_right_y_1 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='mid right y')
        top_left_y_1 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='top left y')
        top_right_y_1 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='top right y')
        top_y_1 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='top y')

        left_x_2 = ipywidgets.FloatSlider(value=0,min=0.0, max=20.0,description='left x')
        right_x_2 = ipywidgets.FloatSlider(value=20,min=0.0, max=20.0,description='right x')
        mid_left_x_2 = ipywidgets.FloatSlider(value=3,min=0.0, max=20.0,description='mid left x')
        mid_right_x_2 = ipywidgets.FloatSlider(value=17,min=0.0, max=20.0,description='mid right x')
        top_left_x_2 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0,description='top left x')
        top_right_x_2 = ipywidgets.FloatSlider(value=13,min=0.0, max=20.0,description='top right x')
        top_x_2 = ipywidgets.FloatSlider(value=10,min=0.0, max=20.0,description='top x')

        left_y_2 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='left y')
        right_y_2 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='right y')
        mid_left_y_2 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='mid left y')
        mid_right_y_2 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='mid right y')
        top_left_y_2 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='top left y')
        top_right_y_2 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='top right y')
        top_y_2 = ipywidgets.FloatSlider(value=5,min=0.0, max=20.0, orientation='vertical',description='top y')

        nr_points_1 = ipywidgets.IntSlider(value=20,min=10, max=100, step=5,description='nr sampled from curve')
        nr_points_2 = ipywidgets.IntSlider(value=20,min=10, max=100, step=5,description='nr sampled from curve')

    elif  model == 'clay_slice_2':
        left_x_1 = ipywidgets.FloatSlider(value=0,min=0.0, max=20.0,description='left x')
        right_x_1 = ipywidgets.FloatSlider(value=20,min=0.0, max=20.0,description='right x')
        mid_left_x_1 = ipywidgets.FloatSlider(value=3,min=0.0, max=20.0,description='mid left x')
        mid_right_x_1 = ipywidgets.FloatSlider(value=17,min=0.0, max=20.0,description='mid right x')
        top_left_x_1 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0,description='top left x')
        top_right_x_1 = ipywidgets.FloatSlider(value=13,min=0.0, max=20.0,description='top right x')
        top_x_1 = ipywidgets.FloatSlider(value=10,min=0.0, max=20.0,description='top x')

        left_y_1 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='left y')
        right_y_1 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='right y')
        mid_left_y_1 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='mid left y')
        mid_right_y_1 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='mid right y')
        top_left_y_1 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='top left y')
        top_right_y_1 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='top right y')
        top_y_1 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='top y')

        left_x_2 = ipywidgets.FloatSlider(value=0,min=0.0, max=20.0,description='left x')
        right_x_2 = ipywidgets.FloatSlider(value=20,min=0.0, max=20.0,description='right x')
        mid_left_x_2 = ipywidgets.FloatSlider(value=3,min=0.0, max=20.0,description='mid left x')
        mid_right_x_2 = ipywidgets.FloatSlider(value=17,min=0.0, max=20.0,description='mid right x')
        top_left_x_2 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0,description='top left x')
        top_right_x_2 = ipywidgets.FloatSlider(value=13,min=0.0, max=20.0,description='top right x')
        top_x_2 = ipywidgets.FloatSlider(value=10,min=0.0, max=20.0,description='top x')

        left_y_2 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='left y')
        right_y_2 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='right y')
        mid_left_y_2 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='mid left y')
        mid_right_y_2 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='mid right y')
        top_left_y_2 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='top left y')
        top_right_y_2 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='top right y')
        top_y_2 = ipywidgets.FloatSlider(value=7,min=0.0, max=20.0, orientation='vertical',description='top y')

        nr_points_1 = ipywidgets.IntSlider(value=20,min=10, max=100, step=5,description='nr sampled from curve')
        nr_points_2 = ipywidgets.IntSlider(value=20,min=10, max=100, step=5,description='nr sampled from curve')


    return left_x_1,right_x_1,mid_left_x_1,mid_right_x_1,top_left_x_1,top_right_x_1,top_x_1,left_x_2,right_x_2,mid_left_x_2,mid_right_x_2,top_left_x_2,top_right_x_2,top_x_2,left_y_1,right_y_1,mid_left_y_1,mid_right_y_1,top_left_y_1,top_right_y_1,top_y_1,left_y_2,right_y_2,mid_left_y_2,mid_right_y_2,top_left_y_2,top_right_y_2,top_y_2,nr_points_1,nr_points_2

# make your clay shape here

### slider_1 is x_slice and slider_2 is y_slice 


#### left : the bottom left control point of NURBS 

#### right : the bottom right control point of NURBS 

#### mid_left: the left control point of NURBS at 33% thickness 

#### mid_right: the right control point of NURBS at 33% thickness 

#### top_left: the left control point of NURBS at 66% thickness 

#### top_right: the right control point of NURBS at 66% thickness 

#### top : top control point 

In [5]:
button = ipywidgets.ToggleButtons(
    options=['clay_slice_1', 'clay_slice_2'],
    description='choose standard model:',
    disabled=False
)
display(button)

ToggleButtons(description='choose standard model:', options=('clay_slice_1', 'clay_slice_2'), value='clay…

In [8]:
out = choose_standard_model(model=button.value)
left_x_1,right_x_1,mid_left_x_1,mid_right_x_1,top_left_x_1,top_right_x_1,top_x_1,left_x_2,right_x_2,mid_left_x_2,mid_right_x_2,top_left_x_2,top_right_x_2,top_x_2,left_y_1,right_y_1,mid_left_y_1,mid_right_y_1,top_left_y_1,top_right_y_1,top_y_1,left_y_2,right_y_2,mid_left_y_2,mid_right_y_2,top_left_y_2,top_right_y_2,top_y_2,nr_points_1,nr_points_2 = out
slider_1 = ipywidgets.interactive_output(NURBS,
                    {"left_x":left_x_1,
                    "right_x":right_x_1,
                    "mid_left_x":mid_left_x_1,
                    "mid_right_x":mid_right_x_1,
                    "top_left_x":top_left_x_1,
                    "top_right_x":top_right_x_1,
                    "top_x":top_x_1,
                    "left_y":left_y_1,
                    "right_y":right_y_1,
                    "mid_left_y":mid_left_y_1,
                    "mid_right_y":mid_right_y_1,
                    "top_left_y":top_left_y_1,
                    "top_right_y":top_right_y_1,
                    "top_y":top_y_1,
                    "nr_points":nr_points_1})
ui_1_x = ipywidgets.VBox([left_x_1,mid_left_x_1,top_left_x_1,top_x_1,top_right_x_1,mid_right_x_1,right_x_1,nr_points_1])
ui_2_x = ipywidgets.HBox([left_y_1,mid_left_y_1,top_left_y_1,top_y_1,top_right_y_1,mid_right_y_1,right_y_1])

ui_x = ipywidgets.HBox([ui_1_x,ui_2_x])
display(ui_x,slider_1)
slider_2 = ipywidgets.interactive_output(NURBS,
                    {"left_x":left_x_2,
                    "right_x":right_x_2,
                    "mid_left_x":mid_left_x_2,
                    "mid_right_x":mid_right_x_2,
                    "top_left_x":top_left_x_2,
                    "top_right_x":top_right_x_2,
                    "top_x":top_x_2,
                    "left_y":left_y_2,
                    "right_y":right_y_2,
                    "mid_left_y":mid_left_y_2,
                    "mid_right_y":mid_right_y_2,
                    "top_left_y":top_left_y_2,
                    "top_right_y":top_right_y_2,
                    "top_y":top_y_2,
                    "nr_points":nr_points_2})
ui_1_y = ipywidgets.VBox([left_x_2,mid_left_x_2,top_left_x_2,top_x_2,top_right_x_2,mid_right_x_2,right_x_2,nr_points_2])
ui_2_y = ipywidgets.HBox([left_y_2,mid_left_y_2,top_left_y_2,top_y_2,top_right_y_2,mid_right_y_2,right_y_2])

ui_y = ipywidgets.HBox([ui_1_y,ui_2_y])
display(ui_y,slider_2)

Output()

Output()

In [7]:
slice_1 = NURBS(left_x_1.value,right_x_1.value,mid_left_x_1.value,mid_right_x_1.value,top_left_x_1.value,top_right_x_1.value,top_x_1.value,
                          left_y_1.value,right_y_1.value,mid_left_y_1.value,mid_right_y_1.value,top_left_y_1.value,top_right_y_1.value,top_y_1.value,
                          nr_points_1.value,fig=False)
slice_2 = NURBS(left_x_2.value,right_x_2.value,mid_left_x_2.value,mid_right_x_2.value,top_left_x_2.value,top_right_x_2.value,top_x_2.value,
                          left_y_2.value,right_y_2.value,mid_left_y_2.value,mid_right_y_2.value,top_left_y_2.value,top_right_y_2.value,top_y_2.value,
                          nr_points_2.value,fig=False)


In [9]:
slice_3 = NURBS(left_x_1.value,right_x_1.value,mid_left_x_1.value,mid_right_x_1.value,top_left_x_1.value,top_right_x_1.value,top_x_1.value,
                          left_y_1.value,right_y_1.value,mid_left_y_1.value,mid_right_y_1.value,top_left_y_1.value,top_right_y_1.value,top_y_1.value,
                          nr_points_1.value,fig=False)
slice_4 = NURBS(left_x_2.value,right_x_2.value,mid_left_x_2.value,mid_right_x_2.value,top_left_x_2.value,top_right_x_2.value,top_x_2.value,
                          left_y_2.value,right_y_2.value,mid_left_y_2.value,mid_right_y_2.value,top_left_y_2.value,top_right_y_2.value,top_y_2.value,
                          nr_points_2.value,fig=False)

In [13]:
orientation_1,surface_1 = clay(slice_1_parameter=slice_1,slice_2_parameter=slice_2,slice_1_pos=10,slice_2_pos=10,formation = 'clay_1').set_input_data()
orientation_2,surface_2 = clay(slice_1_parameter=slice_3,slice_2_parameter=slice_4,slice_1_pos=9,slice_2_pos=10, formation = 'clay_2').set_input_data()

In [14]:
surface_all = pd.concat([surface_1,surface_2],ignore_index=True)
orientation_all = pd.concat([orientation_1,orientation_2],ignore_index=True)

In [15]:
geo_model = gp.create_model('clay')
gp.init_data(geo_model, [0, 20, 0, 20, 0, 20], [50, 50, 50],
                orientations_df = orientation_all,
                surface_points_df = surface_all,
                default_values=True)
geo_model.add_surfaces('basement')
gp.set_interpolator(geo_model,
                    compile_theano=True,
                    theano_optimizer='fast_compile',
                    )
sol = gp.compute_model(geo_model)
gpv = gp.plot_3d(geo_model, image=False, plotter_type='basic',show_data=True)

Active grids: ['regular']
Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                     values
range            34.641016
$C_o$            28.571429
drift equations        [3]


In [16]:
gp.plot_2d(geo_model, image=True, plotter_type='basic',show_data=True)